# Projet 4

In [1]:
import requests
from bs4 import BeautifulSoup
import re

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

Exemple `URL_PAGE2` doit retourner :

<pre>{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
  'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
  'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
</pre>

In [2]:
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"

# We create a custom Exception class to return an exception in case of a bad status code returned by the website
class BadStatusCode(Exception):
     def __init__(self, code):
         super().__init__('Unable to connect, received status code: {}'.format(code))

def get_prices_from_url(url):
    # we request the url
    r = requests.get(url)

    # we raise an Exception if the status code is not 200
    if r.status_code != 200:
        raise BadStatusCode(r.status_code)
    
    # we get our soup from the downloaded content
    soup = BeautifulSoup(r.content)

    prices = {}

    for div_tag in soup.findAll('div', attrs={'class': 'pricing-table'}):
        price = {}
        # title
        price_title = div_tag.find('div', {'class':'pricing-table-header'}).find('h2').text
        # price
        price['price'] = re.search('(\$[0-9]+)', div_tag.find('span', {'class':'pricing-table-price'}).text.strip()).group(0)

        ul = div_tag.find('ul', {'class':'pricing-table-list'}).find_all('li')
        # storage
        price['storage'] = re.search('([0-9]+[MGTP]?B)', ul[3].text).group(0)
        # database
        price['databases'] = int(re.search('([0-9]+)', ul[4].text).group(0))

        # we add this price to the prices dict
        prices[price_title] = price

    return prices

In [3]:
get_prices_from_url(URL_PAGE3)

{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25},
 'Privilege': {'price': '$99', 'storage': '1TB', 'databases': 100}}

Ecrire une fonction qui extrait des informations sur une bière de beowulf.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33 

La fonction doit retourner :
<pre>
{'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33}
</pre>

In [4]:
def extract_beer_infos(url):
    # we request the url
    r = requests.get(url)

    # we raise an Exception if the status code is not 200
    if r.status_code != 200:
        raise BadStatusCode(r.status_code)
    
    # we get our soup from the downloaded content
    soup = BeautifulSoup(r.content)
    
    infos = {
        'name': soup.find('div', {'class': 'product-detail-info-title'}).find('h1').text,
        'note': int(soup.find('div', {'class': 'stars'}).attrs['data-percent']),
        'price': float(soup.find('span', {'class': 'price'}).text[:-2].replace(',', '.')),
        'volume': int(soup.find('dd', {'class': 'js-beer-volume'}).attrs['data-volume'][:-3]),
    }
    return infos

In [5]:
extract_beer_infos('https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33')

{'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33}

Cette URL retourne un JSON avec une liste de bières :

In [6]:
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"

Ecrire une fonction qui prend l'argument cet URL retourne les informations sur une liste de bière via l'API de beowulf.

Cette fonction doit retourner la liste des informations obtenues par la fonction `extract_beer_infos()` définie ci-dessus.

Chercher comment optimiser cette fonction en utilisant multiprocessing.Pool pour paralléliser les accès web.

Exemple de retour :

<pre>[{'name': 'Gallia East IPA', 'note': 80, 'price': 42.99, 'volume': 33},
    {'name': 'La Lager Sans Gluten de Vézelay', 'note': 60, 'price': 38.99, 'volume': 25},
    {'name': 'Brasserie De Sutter Brin de Folie', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'La Cristal IPA du Mont Blanc', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'La Parisienne Le Titi Parisien', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'Gallia Session IPA', 'note': 70, 'price': 42.99, 'volume': 33},
    {'name': 'Ninkasi Brut IPA', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Pietra', 'note': 60, 'price': 38.99, 'volume': 33},
    {'name': 'Desperados', 'note': 60, 'price': 35.99, 'volume': 33},
    {'name': 'Gallia West IPA', 'note': 70, 'price': 42.99, 'volume': 33}]
</pre>

In [7]:
def extract_beer_list_infos(url):
    # Collecter les pages de bières à partir du JSON
    # we request the url
    r = requests.get(url)

    # we raise an Exception if the status code is not 200
    if r.status_code != 200:
        raise BadStatusCode(r.status_code)
    
    # we read our json data
    data = r.json()
    base_url = 'https://www.beerwulf.com'

    # Sequential version (slow):
    beers = [extract_beer_infos(base_url+item['contentReference']) for item in data['items']]

    # Parallel version (faster):
    # beers = []
    
    return beers

In [8]:
from multiprocessing import Pool

def extract_beer_list_infos_parallele(url):
    # Collecter les pages de bières à partir du JSON
    # we request the url
    r = requests.get(url)

    # we raise an Exception if the status code is not 200
    if r.status_code != 200:
        raise BadStatusCode(r.status_code)
    
    # we read our json data
    data = r.json()
    base_url = 'https://www.beerwulf.com'

    # Parallel version (faster):
    # We prepare a list of url to map
    beers_urls = [base_url+item['contentReference'] for item in data['items']]
    # we create a pool to process this in parallele
    with Pool(3) as p:
        beers = p.map(extract_beer_infos, beers_urls)
    
    return beers

In [9]:
# for some reason, this creates an infinit loop in Jupyter
#if __name__ == '__main__':
#    extract_beer_list_infos_parallele(URL_BEERLIST_FRANCE)

In [10]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        # We should have 11 French beers:
        self.assertIsInstance(infos, list)
        self.assertEqual(len(infos), 11)
        # All of them are 25cl or 33cl:
        for beer in infos:
            self.assertIn(beer['volume'], [25, 33])

            
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [11]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Lesson4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Lesson4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 3.342s

OK
